<a href="https://colab.research.google.com/github/11doris/jazz-maestro/blob/colab_word_embeddings/tune_similarity_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip uninstall gensim -y

Found existing installation: gensim 3.6.0
Uninstalling gensim-3.6.0:
  Successfully uninstalled gensim-3.6.0


In [1]:
!pip install gensim

In [2]:
import gensim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pprint
import pandas as pd
import numpy as np
from collections import Counter
import plotly.express as px
from tqdm import tqdm 
from gensim.models.doc2vec import Doc2Vec
import pickle
import os

In [3]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
print(gensim.__version__)

4.1.2


# Initialization

## Download the Data

Major Triads, Minor Triads and Dom7 chords:

In [54]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd' -O input_seq.txt

--2021-11-13 19:32:31--  https://docs.google.com/uc?export=download&id=13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd
Resolving docs.google.com (docs.google.com)... 142.250.72.142, 2607:f8b0:4007:814::200e
Connecting to docs.google.com (docs.google.com)|142.250.72.142|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hl6g6gag7djitrnddtmaam1slbslpl06/1636831950000/14329102864480165501/*/13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd?e=download [following]
--2021-11-13 19:32:32--  https://doc-08-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hl6g6gag7djitrnddtmaam1slbslpl06/1636831950000/14329102864480165501/*/13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd?e=download
Resolving doc-08-4c-docs.googleusercontent.com (doc-08-4c-docs.googleusercontent.com)... 142.250.68.1, 2607:f8b0:4007:80f::2001
Connecting to doc-08-4c-docs.googleusercontent.com (doc-08-4c-docs.googleu

Meta Data:

In [55]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q' -O input_meta.csv

--2021-11-13 19:32:32--  https://docs.google.com/uc?export=download&id=1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q
Resolving docs.google.com (docs.google.com)... 142.250.68.14, 2607:f8b0:4007:814::200e
Connecting to docs.google.com (docs.google.com)|142.250.68.14|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5eesticjdh3vcalqf3t7ncfmla99s4b1/1636831950000/14329102864480165501/*/1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q?e=download [following]
--2021-11-13 19:32:33--  https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5eesticjdh3vcalqf3t7ncfmla99s4b1/1636831950000/14329102864480165501/*/1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q?e=download
Resolving doc-14-4c-docs.googleusercontent.com (doc-14-4c-docs.googleusercontent.com)... 142.250.68.1, 2607:f8b0:4007:80f::2001
Connecting to doc-14-4c-docs.googleusercontent.com (doc-14-4c-docs.googleuse

In [56]:
path_to_file = '/content/input_seq.txt'

with open(path_to_file) as f: 
  lines = f.read().splitlines()

In [75]:
data = [line.split(' ') for line in lines]

### Meta Data

In [58]:
meta = pd.read_csv('input_meta.csv', sep='\t')
meta.columns

Index(['id', 'file_name', 'title', 'composer', 'year', 'year_truncated',
       'tonality', 'tune_key', 'tune_mode', 'structure', 'num_bars',
       'time_signature', 'cycle_fifths_order', 'style', 'musicbrainz_id',
       'wikidata_id', 'wikidata_allmusic', 'wiki_link', 'wikidata_description',
       'lyricist'],
      dtype='object')

In [59]:
titles = meta.loc[:, ['id', 'title', 'tune_mode']].drop_duplicates()
titles.set_index('id', inplace=True)

In [60]:
assert len(data) == len(titles)
print(len(data))

1812


In [61]:
titles_dict = titles.to_dict()
index_to_title = titles_dict['title']
index_to_title[1170]

'These Foolish Things'

In [62]:
title_to_index = {value: index for index, value in index_to_title.items()}
title_to_index['These Foolish Things']

1170

# Visualize Chord N-Grams



In [90]:
def ngrams(tokens, n=2, sep='-'):
    return [sep.join(ngram) for ngram in zip(*[tokens[i:] for i in range(n)])]

In [91]:
def list_to_ngrams(data, n=2, sep='-'):
  out = []
  for row in data:
    out.append(ngrams(row, n, sep))

  return out

In [92]:
data_ngrams = list_to_ngrams(data, 3)

In [66]:
def raw_chords_to_df(tunes):
  tunes_chords = [item for tune in tunes for item in tune]
  counts = Counter(tunes_chords)
  df = pd.DataFrame(counts.items(),
                    columns=['chord', 'count']).sort_values(by='count', ascending=False)

  return df

In [67]:
df_chords = raw_chords_to_df(data_ngrams)
df_chords.head(10)

,chord,count
14,Dm7-G7-C,3344
75,G7-C-C,1630
132,A7-Dm7-G7,1506
37,Em7-A7-Dm7,1132
410,C-C-C,1059
179,D7-G7-C,838
85,G7-G7-C,826
71,Am7-Dm7-G7,663
74,C-G7-C,638
6,Gm7-C7-F,615


In [68]:
df_chords = pd.DataFrame.from_dict(df_chords)
df_chords.sort_values(by=['count'], ascending=False, inplace=True)
df_chords_top = df_chords.query('count > 100')

fig = px.bar(df_chords_top, x='chord', y='count', log_y=True)
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

# List Tunes for Test Data

In [140]:
contrafacts = {
               "26-2": "Confirmation",
               "52nd Street Theme": "I Got Rhythm",
               "Ablution": "All The Things You Are",
               "Anthropology": "I Got Rhythm",
               "Ballade": "As Long As I Live",
               "Bright Mississippi": "Sweet Georgia Brown",
               "C.T.A.": "I Got Rhythm",
               "Celia": "I Got Rhythm",
               "Cottontail": "I Got Rhythm",
               "Countdown": "Tune Up",
               "Dewey Square": "Oh, Lady Be Good",
               "Dexterity": "I Got Rhythm",
               "Dig": "Sweet Georgia Brown",
               "Donna Lee": "Indiana (Back Home Again In)",
               "Don't Be That Way": "I Got Rhythm",
               "Eternal Triangle": "I Got Rhythm",
               "Evidence": "Just You, Just Me",
               "Flintstones": "I Got Rhythm",
               "Four On Six": "Summertime",
               "Freight Train": "Blues For Alice",
               "Good Bait": "I Got Rhythm",
               "Hackensack": "Oh, Lady Be Good",
               "Half Nelson": "Lady Bird",
               "Hot House": "What Is This Thing Called Love",
               "Impressions": "So What",
               "In A Mellow Tone (In A Mellotone)": "Rose Room",
               "In Walked Bud": "Blue Skies",
               "Ko Ko": "Cherokee",
               "Lennie's Pennies": "Pennies From Heaven",   ## Lennie's Pennies is in minor and therefore transposed to Amin... not possible to recognize like that
               # "Let's Call This": "Honeysuckle Rose",
               "Little Rootie Tootie": "I Got Rhythm",
               "Little Willie Leaps": "All God's Chillun Got Rhythm",
               "Lullaby Of Birdland": "Love Me Or Leave Me",
               "Moose The Mooche": "I Got Rhythm",
               "My Little Suede Shoes": "Jeepers Creepers",
               "Oleo": "I Got Rhythm",
               "Ornithology": "How High The Moon",
               "Passport": "I Got Rhythm",
               "Quasimodo (Theme)": "Embraceable You",
               "Rhythm-a-ning": "I Got Rhythm",
               "Room 608": "I Got Rhythm",
               "Salt Peanuts": "I Got Rhythm",
               "Satellite": "How High The Moon",
               "Scrapple From The Apple": "Honeysuckle Rose", # A section
               "Scrapple From The Apple": "I Got Rhythm", # B section
               #"Segment": "I Got Rhythm",
               "Seven Come Eleven": "I Got Rhythm",
               "Shaw 'Nuff": "I Got Rhythm",
               "Theme, The": "I Got Rhythm",
               "Tour De Force": "Jeepers Creepers",
               "Wow": "You Can Depend On Me",
               "Yardbird Suite": "Rosetta",
               # following tunes are not from wikipedia:
               "Sweet Sue, Just You": "Honeysuckle Rose",  # A section
               # "All Of Me": "Pennies From Heaven", # bars 25-28 of All of Me are same as bars 17-20 of Pennies From Heaven, but different key!
}

contrafacts_test = []
for reference, compare in contrafacts.items():
  contrafacts_test.append([title_to_index[reference], title_to_index[compare]])
  

In [70]:
contrafacts_test[:5]

[[0, 251], [3, 1527], [30, 63], [80, 1527], [104, 1372]]

# TF-IDF

## Build Model

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [97]:
tfidf_ngrams = [1, 2, 3, 4]

tfidf_model = {}
for n in tfidf_ngrams:

  data_ngrams = list_to_ngrams(data, n=n)
  data_tfidf = [" ".join(i) for i in data_ngrams]

  tfidf_vectorizer = TfidfVectorizer()
  tfidf_model[n] = tfidf_vectorizer.fit_transform(data_tfidf)

In [98]:
tfidf_model

{1: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15947 stored elements in Compressed Sparse Row format>,
 2: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15947 stored elements in Compressed Sparse Row format>,
 3: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15947 stored elements in Compressed Sparse Row format>,
 4: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15947 stored elements in Compressed Sparse Row format>}

## Test of the Contrafacts tunes

In [114]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2']
for c in tfidf_model.keys():
  cols.append(f"cosval_ngram{c}")
df_eval_tfidf = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
  row = [index_to_title[tune1], 
         index_to_title[tune2]]
  for ngram_size, tfidf_m in tfidf_model.items():
    row.append(cosine_similarity(tfidf_m[tune1], tfidf_m[tune2])[0][0])
  
  df_eval_tfidf.loc[len(df_eval_tfidf)] = row

df_eval_tfidf

,Tune1,Tune2,cosval_ngram1,cosval_ngram2,cosval_ngram3,cosval_ngram4
0,26-2,Confirmation,0.568533,0.564252,0.574305,0.576989
1,52nd Street Theme,I Got Rhythm,0.832439,0.832439,0.825817,0.817865
2,Ablution,All The Things You Are,0.886735,0.877070,0.879810,0.878596
3,Anthropology,I Got Rhythm,0.777413,0.777413,0.784389,0.783179
4,Ballade,As Long As I Live,0.634909,0.643877,0.647502,0.641000
5,Bright Mississippi,Sweet Georgia Brown,0.970778,0.969760,0.968172,0.965051
6,C.T.A.,I Got Rhythm,0.698455,0.695247,0.704302,0.709436
7,Celia,I Got Rhythm,0.505318,0.489912,0.486457,0.473649
8,Cottontail,I Got Rhythm,0.915579,0.915579,0.912508,0.909247
9,Countdown,Tune Up,0.744414,0.725241,0.712231,0.697034


# Doc2Vec Embeddings

## Read and Tag Data

In [116]:
import smart_open

def read_corpus(fname, ngram_size, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = ngrams(line.split(), ngram_size, '-')
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

## Doc2Vec Hyperparameters

In [117]:
!rm -R models_doc2vec
!rm -R traindata_doc2vec

rm: cannot remove 'models_doc2vec': No such file or directory
rm: cannot remove 'traindata_doc2vec': No such file or directory


In [118]:
!mkdir models_doc2vec
!mkdir traindata_doc2vec

In [119]:
model_path = './models_doc2vec'
model_prefix = 'autos'
data_path = './traindata_doc2vec'

param_grid = {'d2v': {'variant': [#'pv-dm', 
                                  'pv-dbow'], 
                      'window': [
                                 #2, 3, 
                                 4, 
                                 #10
                                 ],
                      'sample': [0.1],
                      'ngrams': [
                                 #1, 2, 
                                 3, 
                                 #4
                                 ],
                     }
              }
size = 100
epochs = 50

for algo, params in param_grid.items():
    for variant in params['variant']:
        dm = 1 if variant == 'pv-dm' else 0
        for ngram_size in params['ngrams']:
            
            # prepare data with the provided ngram size
            train_corpus = list(read_corpus(path_to_file, ngram_size=ngram_size))

            # evaluate different window sizes
            for window in params['window']:

                # evaluate the effect of different sample sizes
                for sample in params['sample']:

                    # calculate the model
                    if algo == 'd2v':
                        model = gensim.models.doc2vec.Doc2Vec(vector_size=size, 
                                                              window=window,
                                                              dm = dm,
                                                              min_count=5, 
                                                              sample = sample, # threshold for configuring which higher-frequency words are randomly downsampled
                                                              epochs=epochs,
                                                              )
                        model.build_vocab(train_corpus)
                        model.train(train_corpus, 
                                    total_examples=model.corpus_count, 
                                    epochs=model.epochs)

                    else:
                        model = gensim.models.FastText(data_ngrams, vector_size=size, window=window, sg=sg, epochs=50)

                    file_name = f"{model_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}_{sample}"
                    model.save(file_name + '.model')

                    file_name = f"{data_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}_{sample}"
                    with open(file_name + '.train', 'wb') as f:
                          # Pickle the 'data' dictionary using the highest protocol available.
                          pickle.dump(train_corpus, f, pickle.HIGHEST_PROTOCOL)

2021-11-13 19:58:10,145 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d100,n5,mc5,s0.1,t3)', 'datetime': '2021-11-13T19:58:10.145407', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-13 19:58:10,147 : INFO : collecting all words and their counts
2021-11-13 19:58:10,149 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-11-13 19:58:10,173 : INFO : collected 8596 word types and 1812 unique tags from a corpus of 1812 examples and 79166 words
2021-11-13 19:58:10,175 : INFO : Creating a fresh vocabulary
2021-11-13 19:58:10,194 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=5 retains 1997 unique words (23.23173569101908%% of original 8596, drops 6599)', 'datetime': '2021-11-13T19:58:10.194128', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+

In [120]:
from gensim.models import KeyedVectors

from os import listdir
from os.path import isfile, join

names = [f for f in listdir(model_path) if isfile(join(model_path, f))]
print(names)
models = {}

for name in names:
    file_name = f"{model_path}/{name}"
    print(file_name)
    models[name] = Doc2Vec.load(file_name)

2021-11-13 19:58:23,644 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dbow_3_4_0.1.model
2021-11-13 19:58:23,654 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_4_0.1.model.dv.* with mmap=None
2021-11-13 19:58:23,656 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_4_0.1.model.wv.* with mmap=None
2021-11-13 19:58:23,658 : INFO : setting ignored attribute cum_table to None
2021-11-13 19:58:23,690 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_3_4_0.1.model', 'datetime': '2021-11-13T19:58:23.690233', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}


['autos_d2v_pv-dbow_3_4_0.1.model']
./models_doc2vec/autos_d2v_pv-dbow_3_4_0.1.model


In [121]:
names = [f for f in listdir(data_path) if isfile(join(data_path, f))]
print(names)
data = {}

for name in names:
    file_name = f"{data_path}/{name}"
    print(file_name)
    with open(file_name, 'rb') as f:
        data[name] = pickle.load(f)

['autos_d2v_pv-dbow_3_4_0.1.train']
./traindata_doc2vec/autos_d2v_pv-dbow_3_4_0.1.train


In [122]:
for name, m in models.items():
  print(m)

Doc2Vec(dbow,d100,n5,mc5,s0.1,t3)


In [123]:
def compare_doc_models(models, **kwargs):

    passes = 3

    df = pd.DataFrame()
    names = []
    scores = []
    runs = []

    for p in range(passes):
      print(f"\n*** Run {p}")
      for name, model in models.items():
        ranks = []
        second_ranks = []
        similarities = []

        print(name)

        # load the training corpus data
        train_name = os.path.splitext(name)[0] + '.train'
        train_corpus = data[train_name]
        print(train_corpus[:5])

        # use the model to infer the similarity for a tune
        for doc_id in range(len(train_corpus)):
            inferred_vector = model.infer_vector(train_corpus[doc_id].words)
            sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
            similarities.append(sims)
            rank = [docid for docid, sim in sims].index(doc_id)
            ranks.append(rank)

            second_ranks.append(sims[1])

        counter = Counter(ranks)
        print(counter)

        tunes_sim = counter[0]
        tunes_not_sim = len(train_corpus) - counter[0]
        score = 100 * tunes_not_sim/(tunes_sim+tunes_not_sim)

        print(f"\tTunes similar to themselves in first place: {tunes_sim}")
        print(f"\tTunes not similar in first place: {tunes_not_sim}")
        print(f"\t% of tunes not similar to themselves in first place: {score:.2f}%")

        names.append(name)
        scores.append(score)
        runs.append(p)
      
    df = pd.DataFrame({'name': names, 
                        'score': scores,
                        'pass': runs
                      })

    return df

In [124]:
df_score = compare_doc_models(models)


*** Run 0
autos_d2v_pv-dbow_3_4_0.1.model
[TaggedDocument(words=['C-Eb7-G#', 'Eb7-G#-B7', 'G#-B7-E', 'B7-E-G7', 'E-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-F', 'C7-F-G#7', 'F-G#7-C#', 'G#7-C#-E7', 'C#-E7-Am7', 'E7-Am7-D7', 'Am7-D7-Dm7', 'D7-Dm7-G7', 'Dm7-G7-C', 'G7-C-Eb7', 'C-Eb7-G#', 'Eb7-G#-B7', 'G#-B7-E', 'B7-E-G7', 'E-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-F', 'C7-F-Eb7', 'F-Eb7-G#', 'Eb7-G#-B7', 'G#-B7-E', 'B7-E-G7', 'E-G7-C', 'G7-C-Gm7', 'C-Gm7-C7', 'Gm7-C7-Bm7', 'C7-Bm7-E7', 'Bm7-E7-A', 'E7-A-C7', 'A-C7-F', 'C7-F-Bbm7', 'F-Bbm7-Eb7', 'Bbm7-Eb7-G#', 'Eb7-G#-Dm7', 'G#-Dm7-G7', 'Dm7-G7-C', 'G7-C-Eb7', 'C-Eb7-G#', 'Eb7-G#-B7', 'G#-B7-E', 'B7-E-G7', 'E-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-F', 'C7-F-Eb7', 'F-Eb7-G#', 'Eb7-G#-B7', 'G#-B7-E', 'B7-E-G7', 'E-G7-C'], tags=[0]), TaggedDocument(words=['Am7-Am7-Cm7', 'Am7-Cm7-Cm7', 'Cm7-Cm7-Eb', 'Cm7-Eb-Eb', 'Eb-Eb-Em7', 'Eb-Em7-A7', 'Em7-A7-Dm7', 'A7-Dm7-Dm7', 'Dm7-Dm7-Bm7', 'Dm7-Bm7-Bm7', 'Bm7-Bm7-Bbm7', 'Bm7-Bbm7-Bbm7', 'Bbm7-Bbm7-Fm7', 'Bbm7-Fm7-Fm7', 'Fm7-Fm7-E7'

In [125]:
pd.pivot_table(df_score, index = 'name', values = 'score', columns='pass')

pass,0,1,2
name,,,
autos_d2v_pv-dbow_3_4_0.1.model,2.649007,2.81457,2.704194


In [126]:
import time
t = time.localtime()
timestamp = time.strftime('%Y%m%d_%H%M%S', t)
print(timestamp)

20211113_195921


In [127]:
score_filename = f'score_{timestamp}.csv'
df_score.to_csv(score_filename)

In [128]:
 !zip -r models.zip models_doc2vec/ 

  adding: models_doc2vec/ (stored 0%)
  adding: models_doc2vec/autos_d2v_pv-dbow_3_4_0.1.model (deflated 10%)


In [129]:
from google.colab import files
files.download("models.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [130]:
files.download(score_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Assess the best Model

In [131]:
best_model = 'autos_d2v_pv-dbow_3_4_0.1'

In [132]:
model = models[f"{best_model}.model"]

#### Assess self-similarity

In [133]:
ranks = []
second_ranks = []
similarities = []

train_corpus = data[f"{best_model}.train"]

for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    similarities.append(sims)
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [134]:
import collections

counter = collections.Counter(ranks)
print(counter)

tunes_sim = counter[0]
tunes_not_sim = len(train_corpus) - counter[0]

print()
print(f"Tunes similar to themselves in first place: {tunes_sim}")
print(f"Tunes not similar in first place: {tunes_not_sim}")
print(f"% of tunes not similar to themselves in first place: {100 * tunes_not_sim/(tunes_sim + tunes_not_sim):.2f}%")

Counter({0: 1760, 1: 36, 4: 5, 2: 3, 3: 2, 5: 1, 1026: 1, 7: 1, 1049: 1, 290: 1, 297: 1})

Tunes similar to themselves in first place: 1760
Tunes not similar in first place: 52
% of tunes not similar to themselves in first place: 2.87%


In [135]:
print('Tunes that are similar to other tunes before they are similar to themselves:')
for tune_index, value in enumerate(ranks):
  if value > 0:
    print(f"'{titles.iloc[tune_index, 0]}': {value}")

Tunes that are similar to other tunes before they are similar to themselves:
'Anthropology': 5
'Arise, Her Eyes': 1026
'Au Privave': 2
'Basin Street Blues': 1
'Billie's Bounce': 1
'Blue Monk': 2
'Blue Skies': 1
'Blue Sphere': 1
'Dig': 1
'Eighty One': 4
'Get Happy': 1
'Honeysuckle Rose': 1
'I Can't Give You Anything But Love': 1
'I Got Rhythm': 1
'Impressions': 1
'Ko Ko': 1
'Lazy River': 1
'Misterioso': 1
'Moose The Mooche': 7
'Mr. P.C.': 1
'No Moe': 4
'Now's The Time': 4
'Oleo': 2
'On The Sunny Side Of The Street': 1
'Out Of Nowhere': 1
'Passport': 4
'Pfrancing (No Blues)': 3
'Preacher, The': 1
'Re Person I Knew': 1049
'Salt Peanuts': 1
'Sandu': 1
'Shaw 'Nuff': 4
'Sorcerer, The': 290
'St. Louis Blues': 1
'Struttin' With Some Barbecue': 1
'Sway': 1
'Sweet Georgia Brown': 1
'Theme, The': 3
'Things Ain't What They Used To Be': 1
'Toy Tune': 297
'Walkin'': 1
'You Can Depend On Me': 1
'Basin Street Blues': 1
'Birth Of The Blues, The': 1
'Bye Bye Blues': 1
'C'est Si Bon': 1
'Creole Love Call

#### Determine Similarity of Tunes

In [136]:
assert(len(model.dv.vectors == len(list(title_to_index.keys()))))

In [137]:
tunes_eval_list = [
  'These Foolish Things', 
  'Blue Moon',
  'Hundred Years From Today, A',
  "All God's Chillun Got Rhythm",
  'I Got Rhythm',
  'Bye Bye Blackbird',
  'Old Fashioned Love',
  'Dinah',
  'Honeysuckle Rose'
]

In [138]:
for tune in tunes_eval_list:
  sim = model.dv.most_similar(title_to_index[tune])

  print()
  print(f"'{tune}' is most similar to:")
  for index, value in sim:
    print(f'\t{value:.3f}: {index_to_title[index]}')



'These Foolish Things' is most similar to:
	0.716: Mountain Greenery
	0.712: A Foggy Day
	0.701: Isn't It A Pity
	0.682: More Than You Know
	0.680: They Can't Take That Away From Me
	0.677: I've Told Eve'ry Little Star
	0.675: Love Walked In
	0.674: Blue Room, The
	0.669: You Keep Coming Back Like A Song
	0.666: Blue Room, The

'Blue Moon' is most similar to:
	0.728: Teach Me Tonight
	0.722: It's You Or No One
	0.718: Mambo Inn
	0.695: Jeepers Creepers
	0.674: I Could Write A Book
	0.662: Touch Of Your Lips, The
	0.662: All God's Chillun Got Rhythm
	0.659: I Want To Be Happy
	0.656: My Heart Stood Still
	0.655: At Last

'Hundred Years From Today, A' is most similar to:
	0.635: Chicken
	0.565: Save It Pretty Mama
	0.561: Aged And Mellow Blues
	0.557: That's My Weakness Now
	0.551: Full House
	0.548: Blue Hawaii
	0.544: Float Me Down The River
	0.544: White Cliffs Of Dover, The
	0.539: You Belong To Me
	0.538: I May Be Wrong (But I Think You're Wonderful)

'All God's Chillun Got Rhythm'

#### Evaluate Similarity for Jazz Contrafacts



In [145]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2', 'doc2vec']
df_eval_doc2vec = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
  row = [index_to_title[tune1], 
         index_to_title[tune2]]
  row.append(model.dv.similarity(tune1, tune2))
  
  df_eval_doc2vec.loc[len(df_eval_doc2vec)] = row

df_eval_doc2vec

,Tune1,Tune2,doc2vec
0,26-2,Confirmation,0.471185
1,52nd Street Theme,I Got Rhythm,0.674313
2,Ablution,All The Things You Are,0.790106
3,Anthropology,I Got Rhythm,0.667859
4,Ballade,As Long As I Live,0.380979
5,Bright Mississippi,Sweet Georgia Brown,0.784624
6,C.T.A.,I Got Rhythm,0.655681
7,Celia,I Got Rhythm,0.084740
8,Cottontail,I Got Rhythm,0.752822
9,Countdown,Tune Up,0.668736


##### Compare results from TF-IDF and doc2vec



In [147]:
df_eval_tfidf.merge(df_eval_doc2vec)

,Tune1,Tune2,cosval_ngram1,cosval_ngram2,cosval_ngram3,cosval_ngram4,doc2vec
0,26-2,Confirmation,0.568533,0.564252,0.574305,0.576989,0.471185
1,52nd Street Theme,I Got Rhythm,0.832439,0.832439,0.825817,0.817865,0.674313
2,Ablution,All The Things You Are,0.886735,0.877070,0.879810,0.878596,0.790106
3,Anthropology,I Got Rhythm,0.777413,0.777413,0.784389,0.783179,0.667859
4,Ballade,As Long As I Live,0.634909,0.643877,0.647502,0.641000,0.380979
5,Bright Mississippi,Sweet Georgia Brown,0.970778,0.969760,0.968172,0.965051,0.784624
6,C.T.A.,I Got Rhythm,0.698455,0.695247,0.704302,0.709436,0.655681
7,Celia,I Got Rhythm,0.505318,0.489912,0.486457,0.473649,0.084740
8,Cottontail,I Got Rhythm,0.915579,0.915579,0.912508,0.909247,0.752822
9,Countdown,Tune Up,0.744414,0.725241,0.712231,0.697034,0.668736


##### Verbose Testing Output

In [141]:
similarity_found = 0

for tune1, tune2 in contrafacts.items():
  index1 = title_to_index[tune1]
  index2 = title_to_index[tune2]

  sim = model.dv.similarity(index1, index2)
  most_sim = model.dv.most_similar(index1, topn=5)
  
  # check whether the index2 was found among the topn similar tunes
  for i in most_sim:
    if index2 == i[0]:
      similarity_found += 1

  print()
  print(f"'{tune1}' to '{tune2}': {sim}")
  print(f"'{tune1}' is most similar to:")
  for index, value in most_sim:
    print(f'\t{value:.3f}: {index_to_title[index]}')


print()
num_tunes = len(contrafacts)
print(f"Number of tunes tested: {num_tunes}")
print(f"Number of similarities found: {similarity_found}")
print(f"Result: {100 * similarity_found / num_tunes}%")


'26-2' to 'Confirmation': 0.47118502855300903
'26-2' is most similar to:
	0.751: Satellite
	0.687: Giant Steps
	0.641: Countdown
	0.620: What's New
	0.614: Here's That Rainy Day

'52nd Street Theme' to 'I Got Rhythm': 0.6743128299713135
'52nd Street Theme' is most similar to:
	0.868: I've Got My Fingers Crossed
	0.786: L-O-V-E
	0.754: It's A Good Day
	0.740: Coquette
	0.740: Cherry Pink And Apple Blossom White

'Ablution' to 'All The Things You Are': 0.790106475353241
'Ablution' is most similar to:
	0.790: All The Things You Are
	0.629: Tune Up
	0.605: What's New
	0.604: My Last Affair
	0.598: Days And Nights Waiting

'Anthropology' to 'I Got Rhythm': 0.6678590774536133
'Anthropology' is most similar to:
	0.999: Oleo
	0.998: Passport
	0.998: Shaw 'Nuff
	0.997: No Moe
	0.997: Salt Peanuts

'Ballade' to 'As Long As I Live': 0.3809792399406433
'Ballade' is most similar to:
	0.635: Taking A Chance On Love
	0.634: September In The Rain
	0.621: It's Easy To Remember
	0.618: I've Heard That 

In [142]:
model.dv.similarity(0, 1)

0.3104181

#### Download Model

In [ ]:
from google.colab import files
files.download(f"{model_path}/{best_model}.model")
files.download(f"{data_path}/{best_model}.train")